In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from utils import *
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.filterwarnings("ignore")

import gurobipy as gp

## Data

Data (and base code) for the CFLP_10_10 problem is obtained using the Neur2SP repository. We load the data used for training the NN model.

https://github.com/khalil-research/Neur2SP

In [2]:
data = pd.read_csv('Data/CFLP_10_10_data.csv', index_col=0)
data = data.drop(['10','11','12','13','14','14','15','16','17','18','19'], axis=1)

y = data['obj']
X = data.drop(['obj'], axis=1, inplace=False)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

seed = 0
np.random.seed(seed)
msk = np.random.rand(len(X)) < 0.8
X_train = X[msk]
y_train = y[msk]
X_val = X[~msk]
y_val = y[~msk]


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = scaler.feature_names_in_)
X_mean = scaler.mean_
X_std = scaler.scale_
X_val = pd.DataFrame(scaler.transform(X_val), columns = scaler.feature_names_in_)

y_train = pd.DataFrame(scaler.fit_transform(y_train.values.reshape(-1, 1)), columns = [y.name])
y_mean = scaler.mean_
y_std = scaler.scale_
y_val = pd.DataFrame(scaler.transform(y_val.values.reshape(-1, 1)), columns = [y.name])

## IQNN model loading

We load the previously trained IQNN model, so it can be used as a set of piece-wise lienar constraints for representing the second stage objective distribution.

In [3]:
q = np.linspace(0.01, 0.99, num=50, endpoint=True)
nn_model = FeedforwardIncremental(input_size=X_train.shape[1], hidden_size=64, output_size=len(q), num_hidden_layers=1, dropout_rate=0)
nn_model.load_state_dict(torch.load('Models/CFLP_10_10_model.pt'))
nn_model

FeedforwardIncremental(
  (model): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=64, out_features=50, bias=True)
  )
)

## Optimization problem

Parameters for the CFLP_10_10 problem are obtained also from the Neur2SP repository. 

https://github.com/khalil-research/Neur2SP

In [4]:
with open('Data/inst_f10_c10_r2.0_iss1_bt1_nsp20000_nse5000_sd7.pkl', 'rb') as f:
    data_params = pickle.load(f)

In [12]:
class CFLPProblem(object):
    
    def __init__(self, inst, network, lambda_tradeoff):
        
        self.inst = inst
        self.inst['recourse_costs'] = 2 * np.max([np.max(self.inst['fixed_costs']), np.max(self.inst['trans_costs'])])
        self.network = network
        self.mip = None
        self.lambda_tradeoff = lambda_tradeoff
    
    def _make_extensive_model(self, scenarios, alpha):
        """ Formulates two stage extensive form. """
        demands = scenarios
        n_scenarios = len(scenarios)
        scenario_prob = 1 / n_scenarios
        
        model = gp.Model()
        var_dict = {}
        
        # binary variables for each location
        objective_fs = 0
        for i in range(self.inst['n_facilities']):
            var_name = f"x_in[{i}]"
            var_dict[var_name] = model.addVar(lb=0.0, ub=1.0, vtype="B", name=var_name)
            objective_fs += self.inst['fixed_costs'][i]*var_dict[var_name]
        model.update()
        
        objective_stoch = 0
        # add either continous or binary second stage serving costs
        for s in range(n_scenarios):
            for i in range(self.inst['n_facilities']):
                for j in range(self.inst['n_customers']):
                    var_name = f"y_{i}_{j}_{s}"
                    if self.inst['integer_second_stage']:
                        var_dict[var_name] = model.addVar(lb=0.0, ub=1.0, vtype="B",
                                                            name=var_name)
                    else:
                        var_dict[var_name] = model.addVar(lb=0.0, ub=1.0, vtype="C",
                                                            name=var_name)
                    objective_stoch += self.inst['trans_costs'][i, j] * var_dict[var_name]
        model.update()
        
        # add either continous or binary second stage recourse costs
        for s in range(n_scenarios):
            for j in range(self.inst['n_customers']):
                var_name = f"z_{j}_{s}"
                if self.inst['integer_second_stage']:
                    var_dict[var_name] = model.addVar(lb=0.0, ub=1.0, vtype="B", name=var_name)
                else:
                    var_dict[var_name] = model.addVar(lb=0.0, ub=1.0, vtype="C", name=var_name)
                objective_stoch += self.inst['recourse_costs'] * var_dict[var_name]
        model.update()
        
        # add demand constraints
        for s in range(n_scenarios):
            for j in range(self.inst['n_customers']):
                cons = var_dict[f"z_{j}_{s}"]
                for i in range(self.inst['n_facilities']):
                    cons += var_dict[f"y_{i}_{j}_{s}"]
                model.addConstr(cons >= 1, name=f"d_{j}_{s}")
        
        # capacity constraints
        for s in range(n_scenarios):
            for i in range(self.inst['n_facilities']):
                cons = - self.inst['capacities'][i] * var_dict[f"x_in[{i}]"]
                for j in range(self.inst['n_customers']):
                    cons += demands[s][j] * var_dict[f"y_{i}_{j}_{s}"]
                model.addConstr(cons <= 0, name=f"c_{i}")
        
        # bound tightening constraints
        if self.inst['bound_tightening_constrs']:
            for s in range(n_scenarios):
                for i in range(self.inst['n_facilities']):
                    for j in range(self.inst['n_customers']):
                        model.addConstr(- var_dict[f"x_in[{i}]"] + var_dict[f"y_{i}_{j}_{s}"] <= 0, name=f"t_{i}_{j}_{s}")
        model.update()
        
        # cvar variables and constraints
        var_dict["nu"] = model.addVar(lb=0.0, vtype="C", name='nu')
        for s in range(n_scenarios):
            var_name = f"eta_{s}"
            var_dict[var_name] = model.addVar(lb=0.0, vtype="C", name=var_name)
            model.addConstr(sum(self.inst['recourse_costs'] * var_dict[f"z_{j}_{s}"] for j in range(self.inst['n_customers']))
                            + sum(sum(self.inst['trans_costs'][i, j] * var_dict[f"y_{i}_{j}_{s}"] for j in range(self.inst['n_customers'])) for i in range(self.inst['n_facilities']))
                            - var_dict["nu"] - var_dict[f"eta_{s}"] <= 0, 
                            name=f"eta_cons_{s}")
        model.update()
        
        model.setObjective((1+self.lambda_tradeoff)*objective_fs + scenario_prob*objective_stoch
                            + self.lambda_tradeoff*(var_dict["nu"] + 1/(1-alpha)*sum(scenario_prob*var_dict[f"eta_{s}"] for s in range(n_scenarios))), gp.GRB.MINIMIZE)
        exp_val = scenario_prob*objective_stoch
        cvar_val = var_dict["nu"] + 1/(1-alpha)*sum(scenario_prob*var_dict[f"eta_{s}"] for s in range(n_scenarios))
        model.update()
        
        return model, objective_fs, exp_val, cvar_val
    
    def solve_extensive(self, n_scenarios, alpha=0.9, gap=0.0001, time_limit=600, threads=1, log_dir=None, node_file_start=None,
                        node_file_dir=None, test_seed=0):
        """ Solves the extensive form. """

        def callback(model, where):
            """ Callback function to log time, bounds, and first stage sol. """
            if where == gp.GRB.Callback.MIPSOL:
                self.ef_solving_results['time'].append(model.cbGet(gp.GRB.Callback.RUNTIME))
                self.ef_solving_results['primal'].append(model.cbGet(gp.GRB.Callback.MIPSOL_OBJBST))
                self.ef_solving_results['dual'].append(model.cbGet(gp.GRB.Callback.MIPSOL_OBJBND))
                self.ef_solving_results['incumbent'].append(model.cbGetSolution(model._x ))

        # make extensive form 
        scenarios = self.get_scenarios(n_scenarios, test_seed)
        model, objective_fs, exp_val, cvar_val = self._make_extensive_model(scenarios, alpha)
        
        # get variables for callback
        model.update()
        
        self.ef_solving_results = {'primal': [], 'dual': [], 'incumbent': [], 'time': []}
        ef_fs_vars = []
        for i in range(self.inst['n_facilities']):
            ef_fs_vars.append(model.getVarByName(f"x_in[{i}]"))
        model._x = ef_fs_vars

        # solve two_sp
        if log_dir is not None:
            model.setParam("LogFile", log_dir)
        if node_file_start is not None:
            model.setParam("NodefileStart", node_file_start)
            model.setParam("NodefileDir", node_file_dir)
        model.setParam("MIPGap", gap)
        model.setParam("TimeLimit", time_limit)
        model.setParam("Threads", threads)

        model.optimize(callback)
        return model, objective_fs, exp_val, cvar_val
    
    def gen_surrogate_mip(self):
        #####################################
        #    Initialize model variables     #
        #####################################
        self.mip = gp.Model('mipQ')
        x_in = self.mip.addVars(self.inst['n_facilities'], vtype=gp.GRB.BINARY, name='x_in')
        self.mip.update()
        
        # set objective
        objective_fs = 0
        for i in x_in.keys():
            objective_fs += self.inst['fixed_costs'][i] * x_in[i]
        #self.mip.setObjective((1+lambda_tradeoff)*objective, gp.GRB.MINIMIZE)
        self.mip.update()
        
        first_stage_vars = {k: self.mip.getVarByName(f'x_in[{k}]')
                    for k in range(self.inst['n_facilities'])}
        
        ##############################
        #    Network formulation     #
        ##############################
        
        
        x = {}
        z = {}
        x[0] = {}
        z[0] = {}
        
        x_maxs = {}
        x_mins = {}
        x_maxs[0] = {}
        x_mins[0] = {}
        
        layer_list = nn.ModuleList([])
        for layer in self.network.model.children():
            if isinstance(layer, nn.Linear):
                layer_list.append(layer)
        
        l_0 = layer_list[0]
        for i in range(l_0.in_features):
            x[0][i] = (first_stage_vars[i] - X_mean[i])/X_std[i]
            x_maxs[0][i] = (1 - X_mean[i])/X_std[i]
            x_mins[0][i] = (0 - X_mean[i])/X_std[i]
        self.mip.update()
        
        for ind, layer in enumerate(layer_list):
            l = layer
            
            #x[ind+1][i] tracks the output of node i of layer ind
            #z[ind+1][i] is the binary defining whether node i of layer ind is active
            x[ind+1] = {}
            z[ind+1] = {}
            
            x_maxs[ind+1] = {}
            x_mins[ind+1] = {}
            
            for i in range(l.out_features):
                #get weights and biases
                m = l.weight.detach().numpy()[i]
                b = l.bias.detach().numpy()[i]
                
                #compute upper and lower bounds
                ub = sum(x_maxs[ind][j] * max(0,m[j]) + x_mins[ind][j] * min(0,m[j]) for j in range(l.in_features)) + b
                lb = sum(x_mins[ind][j] * max(0,m[j]) + x_maxs[ind][j] * min(0,m[j]) for j in range(l.in_features)) + b
                x_maxs[ind+1][i] = ub
                x_mins[ind+1][i] = lb
                
                if ind < len(layer_list) - 1:
                    # Define vars
                    x[ind+1][i] = self.mip.addVar(0, max(0,ub), name='x_' + str(ind+1) + '_' + str(i))
                    z[ind+1][i] = self.mip.addVar(0, 1, vtype=gp.GRB.BINARY, name='z_' + str(ind+1) + '_' + str(i))
                    # Big-M representation of node i
                    self.mip.addConstr(x[ind+1][i] >= sum(x[ind][j] * m[j] for j in range(l.in_features)) + b)
                    self.mip.addConstr(x[ind+1][i] <= sum(x[ind][j] * m[j] for j in range(l.in_features)) + b - lb*(1-z[ind+1][i]))
                    self.mip.addConstr(x[ind+1][i] <= ub * z[ind+1][i])
                else:
                    if i == 0:
                        x[ind+1][i] = self.mip.addVar(lb=lb, ub=ub, name='x_' + str(ind+1) + '_' + str(i))
                        self.mip.addConstr(x[ind+1][i] == sum(x[ind][j] * m[j] for j in range(l.in_features)) + b)
                    else:
                        x[ind+1][i] = self.mip.addVar(0, max(0,ub), name='x_' + str(ind+1) + '_' + str(i))
                        z[ind+1][i] = self.mip.addVar(0, 1, vtype=gp.GRB.BINARY, name='z_' + str(ind+1) + '_' + str(i))
                        self.mip.addConstr(x[ind+1][i] >= sum(x[ind][j] * m[j] for j in range(l.in_features)) + b)
                        self.mip.addConstr(x[ind+1][i] <= sum(x[ind][j] * m[j] for j in range(l.in_features)) + b - lb*(1-z[ind+1][i]))
                        self.mip.addConstr(x[ind+1][i] <= ub * z[ind+1][i])
            
            self.mip.update()
            
        q_vars = {}
        for i in range(l.out_features):
            # Saving final incremental quantiles
            q_vars[i] = self.mip.addVar(lb=-gp.GRB.INFINITY, name='q_vars_' + str(i))
            if i == 0:
                self.mip.addConstr(q_vars[i] == x[ind+1][i])
            else:
                self.mip.addConstr(q_vars[i] == x[ind+1][i] + q_vars[i-1])
        self.mip.update()
        
        q_distd = {}
        objective_cvar = 0
        objective_mean = 0
        for i in range(l.out_features):
            q_distd[i] = self.mip.addVar(lb=-gp.GRB.INFINITY, name='q_vars_distd_' + str(i))
            self.mip.addConstr(q_distd[i] == q_vars[i]*y_std[0] + y_mean[0])
            if i < 35: # This index indicates the quantile of the distribution. 35th quantile represents VaR 70
                objective_mean += q_distd[i]
            else:
                objective_mean += q_distd[i]
                objective_cvar += q_distd[i]
        self.mip.update()
        
        self.mip.setObjective((1+self.lambda_tradeoff)*objective_fs + (1/50)*objective_mean + (self.lambda_tradeoff/15)*objective_cvar, gp.GRB.MINIMIZE)
        self.mip.update()
    
    def optimize(self, gap=0.0001, threads=8):
        self.mip.setParam('Threads', threads)
        self.mip.setParam('MIPGap', gap)
        self.mip.update()
        
        self.mip.optimize()
    
    def get_scenarios(self, n_scenarios, test_seed):
        rng = np.random.RandomState()
        rng.seed(n_scenarios + test_seed)
        scenarios = [] 
        for _ in range(n_scenarios):
            scenarios.append(rng.randint(5, 35 + 1, size=self.inst['n_customers']))

        return scenarios
    
    def evaluate_sol(self, sol, n_scenarios, alpha, test_seed):
        # We can run the extensive model with fixed first-stage decision variables to check the quality of the solution
        scenarios = self.get_scenarios(n_scenarios, test_seed)
        model, objective_fs, exp_val, cvar_val = self._make_extensive_model(scenarios, alpha)
        model.update()
        
        for ind, var in sol.items():
            model.getVarByName(var.VarName).lb = np.abs(np.round(var.X))
            model.getVarByName(var.VarName).ub = np.abs(np.round(var.X))
        model.update()
        
        model.setParam("OutputFlag", 0)
        model.setParam('Threads', 8)
        model.setParam('MIPGap', 0.0005)
        model.update()
        
        model.optimize()
        obj = model.getObjective()
        
        return obj.getValue(), objective_fs.getValue(), exp_val.getValue(), cvar_val.getValue()

In [13]:
cflp = CFLPProblem(data_params, nn_model, lambda_tradeoff=0.5)
cflp.gen_surrogate_mip()
cflp.optimize()

Set parameter Threads to value 8
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 8 threads

Optimize a model with 440 rows, 337 columns and 8431 nonzeros
Model fingerprint: 0xc9a8ebee
Variable types: 214 continuous, 123 integer (123 binary)
Coefficient statistics:
  Matrix range     [5e-06, 8e+03]
  Objective range  [2e-02, 2e+03]
  Bounds range     [7e-03, 7e+00]
  RHS range        [2e-16, 8e+03]
Presolve removed 401 rows and 301 columns
Presolve time: 0.00s
Presolved: 39 rows, 36 columns, 270 nonzeros
Variable types: 13 continuous, 23 integer (23 binary)
Found heuristic solution: objective 41516.657267
Found heuristic solution: objective 16218.906859

Root relaxation: objective 8.015011e+03, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

First-stage heuristic decisions are obtained.

In [14]:
first_stage_vars = {k: cflp.mip.getVarByName(f'x_in[{k}]')
                for k in range(data_params['n_facilities'])}
first_stage_vars

{0: <gurobi.Var x_in[0] (value 0.0)>,
 1: <gurobi.Var x_in[1] (value 0.0)>,
 2: <gurobi.Var x_in[2] (value 0.0)>,
 3: <gurobi.Var x_in[3] (value 1.0)>,
 4: <gurobi.Var x_in[4] (value 1.0)>,
 5: <gurobi.Var x_in[5] (value 1.0)>,
 6: <gurobi.Var x_in[6] (value 0.0)>,
 7: <gurobi.Var x_in[7] (value 1.0)>,
 8: <gurobi.Var x_in[8] (value 0.0)>,
 9: <gurobi.Var x_in[9] (value 1.0)>}

We perform a small evaluation of the heuristic solution quality.

In [15]:
true_obj = 0
fs_obj = 0
exp_obj = 0
cvar_obj = 0
for i in range(10):
    obj, fs, exp, cvar = cflp.evaluate_sol(first_stage_vars, 500, 0.7, i)
    true_obj += obj
    fs_obj += fs
    exp_obj += exp 
    cvar_obj += cvar
true_obj = true_obj/10
fs_obj = fs_obj/10
exp_obj = exp_obj/10
cvar_obj = cvar_obj/10

In [16]:
print('Weighted objective:', true_obj)
print('First-stage cost:', fs_obj)
print('Second-stage expected value:', exp_obj)
print('Second-stage CVaR value:', cvar_obj)

Weighted objective: 11178.31185622652
First-stage cost: 6235.0
Second-stage expected value: 879.8312765035223
Second-stage CVaR value: 1891.9611594445446


## Extensive form

We could run the extensive form of the optimization problem, which would take a longer time.

In [ ]:
cflp = CFLPProblem(data_params, nn_model, lambda_tradeoff=0.5)
ext_model, objective_fs, exp_val, cvar_val = cflp.solve_extensive(n_scenarios=500, alpha=0.7, threads=8, time_limit=7200)

In [ ]:
print('Weighted objective:', ext_model.getObjective().getValue())
print('First-stage cost:', objective_fs.getValue())
print('Second-stage expected value:', exp_val.getValue())
print('Second-stage CVaR value:', cvar_val.getValue())